using input:
- 1m dem
- 10m dem
- 30m dem
- merit dem

visualize:
- elevation
- hillshades
- slopes
- mTPI
- CHILI

In [1]:
import ee
ee.Initialize()

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import scipy
import folium
import shapely
from IPython.display import Image
import geopandas as gpd
import rasterio
import pygeohydro
from geemap.plot import geopandas_to_ee
import geemap.foliumap as geemap

In [3]:
meadows_df = gpd.read_parquet("./raw_data/ucdmeadows.parquet")
study_area_huc12 = '180400090505'
#study_area_huc12 = '180201210605'
wbd = pygeohydro.watershed.WBD('huc12')
data = wbd.byids(field='huc12', fids=[study_area_huc12])
meadows = meadows_df.loc[meadows_df['HUC12'].isin([study_area_huc12])]
study_area_ee = geopandas_to_ee(data)
meadows_ee = geopandas_to_ee(meadows)

In [55]:
usgs_1m = ee.ImageCollection("USGS/3DEP/1m").filterBounds(study_area_ee).mosaic().clip(study_area_ee)
usgs_10m = ee.Image("USGS/3DEP/10m").clip(study_area_ee)
usgs_30m = ee.Image("USGS/SRTMGL1_003").clip(study_area_ee)
merit = ee.Image("MERIT/DEM/v1_0_3").clip(study_area_ee)

usgs_1m_terrain = ee.Terrain.products(usgs_1m)
usgs_10m_terrain = ee.Terrain.products(usgs_10m)
usgs_30m_terrain = ee.Terrain.products(usgs_30m)
merit_terrain = ee.Terrain.products(merit)

In [56]:
# vis params
elevation_params = {
    'min': 500,
    'max': 2000,
    'palette': [
    '000000', '478fcd', '86c58e', 'afc35e',
    '8f7131', 'b78d4f', 'e2b8a6', 'ffffff']
}

slope_params = {
    'min': 0,
    'max': 90,
    'palette': ['000000', 'ffffff']
}

aspect_params = {
    'min': 0,
    'max': 360,
    'palette': ['000000', 'ffffff']
}

hillshade_params = {
    'min': 0,
    'max': 255,
    'palette': ['000000', 'ffffff']
}

In [64]:
Map = geemap.Map()
Map.centerObject(study_area_ee, 12)

# elevation
#Map.addLayer(usgs_1m, elevation_params, 'USGS 1m')
#Map.addLayer(usgs_10m, elevation_params, 'USGS 10m')
#Map.addLayer(usgs_30m, elevation_params, 'USGS 30m')
#Map.addLayer(merit, elevation_params, 'MERIT')

# hillshade
Map.addLayer(usgs_1m_terrain.select('hillshade'), hillshade_params, 'USGS 1m hillshade')
#Map.addLayer(usgs_10m_terrain.select('hillshade'), hillshade_params, 'USGS 10m hillshade')
Map.addLayer(usgs_30m_terrain.select('hillshade'), hillshade_params, 'USGS 30m hillshade')
#Map.addLayer(merit_terrain.select('hillshade'), hillshade_params, 'MERIT hillshade')


Map.addLayer(meadows_ee, {}, 'meadows')
Map.addLayerControl()

In [65]:
Map

In [63]:
usgs_1m_terrain.select('hillshade').getInfo()

{'type': 'Image',
 'bands': [{'id': 'hillshade',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}